# AI Assignment 2: CLIP zero-shot prediction
## Testing The Ethnic Bias of Clip

## <span style="color: blue;">Basic Imports
----

In [1]:
import os
# import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# from torchvision.datasets import CIFAR10
from torchvision.datasets import STL10
from torchvision import datasets
from torchvision import transforms
import torchvision

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from PIL import Image
from clip import clip

from torch.utils.data import Subset, DataLoader

## <span style="color: blue;">Hyperparameters
----

In [2]:
# Training
BATCH_SIZE = 64
# CLIP
VISUAL_BACKBONE = 'ViT-B/16' # RN50, ViT-B/32, ViT-B/16


## <span style="color: blue;">Device
----

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## <span style="color: blue;">Dataset
----


#### <span style="color: red;">Note: the train datas are not used for Clip, they were use for RESNET to train

#### CIFAR10

In [4]:
# transform_cifar10_test = transforms.Compose([
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=(224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# test_set = torchvision.datasets.CIFAR10(root='/data/lab/data', train=False,
#                                        download=True, transform=transform_cifar10_test)
# train_set = torchvision.datasets.CIFAR10(root='/data/lab/data', train=True,
#                                        download=True, transform=transform_cifar10_test)

# # ########################
# # test_set = Subset(test_set, range(5000))##5000
# train_set = Subset(train_set, range(5000))##5000
# # #######################


# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)
# train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)

# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# dataset_name = 'CIFAR10'

# data_loader_size = len(test_dataloader.dataset)
# print(f"Total number of samples in the CIFAR10: {data_loader_size}")

# class_less = True # mark the amount of class number(less or equal than 10 is marked True)

#### CIFAR100

In [5]:
# ## CIFAR100
# transform_cifar100_test = transforms.Compose([
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=(224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# test_set = torchvision.datasets.CIFAR100(root='/data/lab/data', train=False,
#                                        download=True, transform=transform_cifar100_test)
# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)

# train_set = torchvision.datasets.CIFAR100(root='/data/lab/data', train=True,
#                                        download=True, transform=transform_cifar100_test)
# train_set = Subset(train_set, range(5000))##5000

# train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)

# class_names = [
#     'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
#     'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
#     'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
#     'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard',
#     'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
#     'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
#     'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
#     'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar',
#     'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
#     'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
# ]

# dataset_name = 'CIFAR100'

# data_loader_size = len(test_dataloader.dataset)
# print(f"Total number of samples in the CIFAR100: {data_loader_size}")

# class_less = False # mark the amount of class number(less or equal than 10 is marked True)

#### STL10

In [6]:
# ## STL10
# transform_cifar100_test = transforms.Compose([
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=(224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# test_set = torchvision.datasets.STL10(root='/data/lab/data', split='test',
#                                        download=True, transform=transform_cifar100_test)
# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)

# train_set = torchvision.datasets.STL10(root='/data/lab/data', split='train',
#                                        download=True, transform=transform_cifar100_test)
# train_set = Subset(train_set, range(5000))##5000

# train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)

# class_names = ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']


# dataset_name = 'STL10'

# data_loader_size = len(test_dataloader.dataset)
# print(f"Total number of samples in the STL10: {data_loader_size}")

# class_less = True # mark the amount of class number(less or equal than 10 is marked True)

#### Stanford dogs

In [7]:
# ##stanford dogs
# transform_stanford_dogs_test = transforms.Compose([
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Replace 'root_directory' with the path to the directory containing the Stanford Dogs dataset
# test_set = datasets.ImageFolder(root='/data/lab/data/Images', transform=transform_stanford_dogs_test)
# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# # List of class names for the Stanford Dogs dataset (based on the breed names)
# class_names = test_set.classes

# dataset_name = 'Stanford Dogs'

# data_loader_size = len(test_dataloader.dataset)
# print(f"Total number of samples in the Stanford Dogs dataset: {data_loader_size}")

# class_less = False # mark the amount of class number(less or equal than 10 is marked True)

#### <span style="color: red;">since stanford dogs has no test/train different sets. It needs to be part by the following code

In [8]:
# ##stanford dogs for RESNET18
# transform_stanford_dogs = transforms.Compose([
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# root_directory = '/data/lab/data/Images'
# full_dataset = datasets.ImageFolder(root=root_directory, transform=transform_stanford_dogs)

# # Define the ratio of train and test split
# train_ratio = 0.8
# test_ratio = 1 - train_ratio

# # Calculate the sizes of train and test sets
# train_size = int(train_ratio * len(full_dataset))
# test_size = len(full_dataset) - train_size

# # Split the dataset into train and test sets
# train_set, test_set = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# # Create DataLoaders for train and test sets
# train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
# test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# # List of class names for the Stanford Dogs dataset (based on the breed names)
# class_names = full_dataset.classes

# num_classes = len(class_names)

# dataset_name = 'Stanford Dogs'

# data_loader_size = len(test_dataloader.dataset)
# print(f"Total number of samples in the Stanford Dogs dataset: {data_loader_size}")

# class_less = False # mark the amount of class number(less or equal than 10 is marked True)

#### FashionMNIST

In [9]:
# transform_fashionmnist_test = transforms.Compose([
#     transforms.Grayscale(num_output_channels=3),
#     transforms.Resize(size=224),
#     transforms.CenterCrop(size=(224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))  # Use FashionMNIST mean and std

# ])

# test_set = torchvision.datasets.FashionMNIST(root='/data/lab/data', 
#                                              download=True, 
#                                              train=False,  # Set train=False for the test set
#                                              transform=transform_fashionmnist_test)
# test_dataloader = torch.utils.data.DataLoader(test_set, 
#                                               batch_size=BATCH_SIZE,
#                                               shuffle=False, 
#                                               num_workers=2)
# train_set = torchvision.datasets.FashionMNIST(root='/data/lab/data', train=True,
#                                        download=True, transform=transform_fashionmnist_test)
# train_set = Subset(train_set, range(5000))##5000

# train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
#                                          shuffle=False, num_workers=2)

# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# dataset_name = 'FashionMNIST'

# data_loader_size = len(test_dataloader.dataset)
# print(f"Total number of samples in the FashionMNIST test set: {data_loader_size}")

# class_less = len(class_names) <= 10  # Check if the number of classes is 10 or less


#### UTKFace-ethnic research
#### <span style="color: red;">note: the change of ethnicity groups has to be by hand, change the input path last number from 0 to 4 accordingly

In [10]:
##UTKFace for clip
transform_stanford_dogs_test = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Replace 'root_directory' with the path to the directory containing the Stanford Dogs dataset
test_set = datasets.ImageFolder(root='/data/lab/data/UTKFace_eth_age/ethnicity_4', transform=transform_stanford_dogs_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# List of class names for the Stanford Dogs dataset (based on the breed names)
class_names = test_set.classes
# class_names = [int(class_name.split('_')[1]) for class_name in class_names]
print(class_names)

dataset_name = 'UTKFace'

data_loader_size = len(test_dataloader.dataset)
print(f"Total number of samples in the UTKFace dataset: {data_loader_size}")

class_less = True # mark the amount of class number(less or equal than 10 is marked True)

['age_1', 'age_10', 'age_11', 'age_12', 'age_13', 'age_14', 'age_15', 'age_16', 'age_17', 'age_18', 'age_19', 'age_2', 'age_20', 'age_21', 'age_22', 'age_23', 'age_24', 'age_25', 'age_26', 'age_27', 'age_28', 'age_29', 'age_3', 'age_30', 'age_31', 'age_32', 'age_33', 'age_34', 'age_35', 'age_36', 'age_37', 'age_38', 'age_39', 'age_4', 'age_40', 'age_41', 'age_42', 'age_43', 'age_44', 'age_45', 'age_46', 'age_47', 'age_48', 'age_49', 'age_5', 'age_50', 'age_51', 'age_52', 'age_53', 'age_54', 'age_55', 'age_56', 'age_57', 'age_58', 'age_59', 'age_6', 'age_60', 'age_63', 'age_64', 'age_65', 'age_66', 'age_69', 'age_7', 'age_75', 'age_76', 'age_8', 'age_80', 'age_82', 'age_84', 'age_85', 'age_9']
Total number of samples in the UTKFace dataset: 1715


In [11]:
##UTKFace for RESNET18
transform_stanford_dogs = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
root_directory = '/data/lab/data/UTKFace_folded/e_b_folded_num'
full_dataset = datasets.ImageFolder(root=root_directory, transform=transform_stanford_dogs)

# Define the ratio of train and test split
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Calculate the sizes of train and test sets
train_size = int(train_ratio * len(full_dataset))
test_size = len(full_dataset) - train_size

# Split the dataset into train and test sets
train_set, test_set = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Create DataLoaders for train and test sets
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# List of class names for the Stanford Dogs dataset (based on the breed names)
class_names = full_dataset.classes

num_classes = len(class_names)

dataset_name = 'UTKFace_age'

data_loader_size = len(test_dataloader.dataset)
print(f"Total number of samples in the UTKFace dataset: {data_loader_size}")

class_less = False # mark the amount of class number(less or equal than 10 is marked True)

Total number of samples in the UTKFace dataset: 2400


## <span style="color: blue;">Model
----

#### Clip

In [12]:
# Load the model
model, preprocess = clip.load(name=VISUAL_BACKBONE, device=device, download_root='/shareddata/clip/')
model = model.to(device)

#### ResNet18
#### including training

In [13]:
# model = torchvision.models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features

# # 替换模型最后的全连接层，假设我们要分类10个类别
# model.fc = nn.Linear(num_ftrs, len(class_names))

# # 将模型移至GPU（如果可用）
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # 定义损失函数和优化器
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# # 训练模型
# for epoch in range(20):  # 假设训练5个epoch
#     running_loss = 0.0
#     for i, data in enumerate(train_dataloader, 0):
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 100 == 99:    # 每100个batch输出一次损失
#             print(f'Epoch [{epoch + 1}, {i + 1}], Loss: {running_loss / 100:.4f}')
#             running_loss = 0.0

# print('Finished Training')

# # 保存训练好的模型
# torch.save(model.state_dict(), 'resnet_cifar10.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "/opt/pytorch/pytorch/c10/cuda/CUDACachingAllocator.cpp":1150, please report a bug to PyTorch. 

## <span style="color: blue;">Prompt Gereration
---

Please denfine a function named ``prompt_encode`` to encode the text using CLIP text encoder.


In [ ]:
prompt = 'a photo of a' # you can try different prompt

def prompt_encode(prompt):
    """
    Args:
        prompt (str): the text prefix before the class

    Returns:
        text_inputs(torch.Tensor)

    """
    #################### Write your answer here ##################
# #use in datasets excepts for the remaining
#     text_inputs = torch.cat([clip.tokenize(f"{prompt} {c}" for c in class_names)]).to(device)

# #use in FashionMNIST
#     text_inputs = torch.cat([clip.tokenize(f"{prompt} {c}, a kind of fashion item" for c in class_names)]).to(device)
    
# #use in Stanford dogs
#     text_inputs = torch.cat([clip.tokenize(f"{prompt} {c}, a type of dog." for c in class_names)]).to(device)

#use in UTKFace
    text_inputs = torch.cat([clip.tokenize(f"{prompt} {c} years old person" for c in class_names)]).to(device)
        ###############################################################
    
    return text_inputs


## <span style="color: blue;">Zero-shot inference


In [ ]:
def model_inference(model, image, text_inputs):

    ##################### Write your answer here ##################
    image = image.to(device)  # Assuming 'image' is a single-channel image
    
    # If the model expects a three-channel input (RGB), expand the single channel to three
    if image.shape[1] == 1:
        image = image.expand(-1, 3, -1, -1)  # Expand single channel to three channels (RGB)
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    ###############################################################
    
    return similarity

##  <span style="color: blue;"> Zero-shot accuracy calculation
---

### Clip Normal cases 

In [ ]:
# testing_loss = []
# testing_acc = []
# correct = 0
# total = 0
# t = 0

# with torch.no_grad():
#     model.eval()
#     text_inputs = prompt_encode(prompt)

#     ##################### Write your answer here ##################
#     text_features = model.encode_text(text_inputs)
#     text_features /= text_features.norm(dim = 1, keepdim = True)
    
#     correct = 0
#     val_corrects = 0
    
#     for batch_idx, (image, target) in enumerate(test_dataloader):
#         t += 1
#         target = target.to(device)
#         image = image.to(device)
        
#         similarity = model_inference(model, image, text_inputs)
#         values, indices = similarity.max(-1)
#         logits = indices
        

#         total += target.size(0)
#         correct += (logits == target).sum().item()
        
#     val_acc = correct/total
        
#      ###############################################################
#     print(t)
#     print(f"the zero-shot performance on {dataset_name} is {val_acc*100:.2f}%, visual encoder is {VISUAL_BACKBONE}.")
    

### Clip With top-k 

In [ ]:
# ##top-k accuracy, don't use in small classes
# top_k = 1  # Change this value to your desired top-k
# total = 0

# training_loss = []
# training_acc = []
# testing_loss = []
# testing_acc = []

# with torch.no_grad():
#     model.eval()
#     text_inputs = prompt_encode(prompt)

#     text_features = model.encode_text(text_inputs)
#     text_features /= text_features.norm(dim=1, keepdim=True)
    
#     correct = 0
#     correct_topk = 0  # Track correct predictions in top-k
    
#     for batch_idx, (image, target) in enumerate(test_dataloader):
#         target = target.to(device)
#         image = image.to(device)

#         logits = model_inference(model, image, text_inputs)
        
#         _, pred = logits.topk(top_k, dim=1)  # Get top-k predictions

#         total += target.size(0)

#         # Check if the target is among the top-k predictions for each sample
#         if(not class_less):
#             for i in range(target.size(0)):
#                 if target[i] in pred[i]:
#                     correct_topk += 1

#         # Check if the top-1 prediction matches the target
#         correct += (pred[:, 0] == target).sum().item()  

#     val_acc = correct / total
    
#     if(not class_less):
#         val_topk_acc = correct_topk / total  # Calculate top-k accuracy


# with open('performance_results.txt', 'a') as file:
#         file.write(f'---------------------------------------------\n')
#         file.write(f'More specific prompt "a type of dog"\n')
#         file.write(f"Dataset: {dataset_name}\n")
#         file.write(f"Zero-shot performance: {val_acc * 100:.2f}%\n")
#         if(not class_less):
#             file.write(f"Top-{top_k} accuracy: {val_topk_acc * 100:.2f}%\n\n")
        
# print(f"Zero-shot performance on {dataset_name} is {val_acc * 100:.2f}%")
# if(not class_less):
#     print(f"Top-{top_k} accuracy on {dataset_name} is {val_topk_acc * 100:.2f}%")


### Clip for UTKFace

In [ ]:
top_k = 1  # Change this value to your desired top-k
total = 0

# Function to classify age into 15-year intervals
def classify_age(age):
    return (age // 20) * 20

with torch.no_grad():
    model.eval()
    text_inputs = prompt_encode(prompt)

    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=1, keepdim=True)
    
    correct = 0
    age_group_correct = {}  # Track correct predictions within age groups

    for batch_idx, (image, target) in enumerate(test_dataloader):
        target = target.to(device)
        image = image.to(device)

        logits = model_inference(model, image, text_inputs)
        _, pred = logits.topk(top_k, dim=1)  # Remove this line if not needed

        total += target.size(0)

        correct += (abs(pred[:, 0] - target) <= 5).sum().item() # count as correct within 5 years

    val_acc = correct / total

    # Calculate accuracy within age groups
    age_group_acc = {age_group: age_group_correct.get(age_group, 0) / total for age_group in age_group_correct}

with open('ethnicity_age_results.txt', 'a') as file:
    file.write(f"-------------------ViT-B/16----------------------############\n")
    file.write(f"age range = 6\n")
    file.write(f"ethnicity = All\n")
    file.write(f"Dataset: {dataset_name}\n")
    file.write(f"Datasize: {data_loader_size}\n")
    file.write(f"Zero-shot performance: {val_acc * 100:.2f}%\n")
        
print(f"Zero-shot performance on {dataset_name} is {val_acc * 100:.2f}%")



### RESNET

In [ ]:
##top-k accuracy, don't use in small classes
top_k = 5  # Change this value to your desired top-k
total = 0


training_loss = []
training_acc = []
testing_loss = []
testing_acc = []

with torch.no_grad():
    model.eval()
    text_inputs = prompt_encode(prompt)

#     correct = 0
#     correct_topk = 0  # Track correct predictions in top-k
    
    val_loss = 0.0
    val_corrects = 0
    
    for batch_idx, (image, target) in enumerate(test_dataloader):
        target = target.to(device)
        image = image.to(device)
        
##########################
        output = model(image)
        
        _, preds = torch.max(output, 1)
#         print(preds)
#         print(target.data)
            
        val_corrects += torch.sum(abs(preds - target.data) <= 5)

    val_acc = val_corrects.double() / len(test_set)
    print(f'Acc: {val_acc:.4f}')
##########################

with open('RESNET18_ethnic.txt', 'a') as file:
        file.write(f'---------------------------------------------\n')
        file.write(f"Dataset: {dataset_name}\n")
        file.write(f"ethnicity: White\n")
        file.write(f"Zero-shot performance: {val_acc * 100:.2f}%\n")
        
#         if(not class_less):
#             file.write(f"Top-{top_k} accuracy: {val_topk_acc * 100:.2f}%\n\n")
        
print(f"Zero-shot performance on {dataset_name} is {val_acc * 100:.2f}%")
# if(not class_less):
#     print(f"Top-{top_k} accuracy on {dataset_name} is {val_topk_acc * 100:.2f}%")
